# Code

## Imports

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import plotly.io as pio

## 1. Gradientenverfahren 1D

#### 1.0.1 Define function and the derivative

In [ ]:
# Funktion definieren
def fn1d(x: float) -> float:
    return x**5 - 3 * x**3 + x**2 + 2


# Händisch Ableitung bestimmen
def fn1d_prime(x: float) -> float:
    return 5 * x**4 - 9 * x**2 + 2 * x

#### 1.0.2 Gradient Descent block

In [ ]:
# Lokales Minimum finden mit Gradientenverfahren
# Parameters
x_start_d1 = 2  # Startpunkt
lr = 1e-2  # Schrittgröße (aka. Learning Rate) -> smaller values lead to an increase in accuracy
num_rep_1d = 10  # number of repetetive checks before the gradientt descent stops -> increase in number is proportional to accuracy
grad_diff = fn1d_prime(x_start_d1)  # average change over the last num_rep_1d
change_1d = True
ys = []
xs = []

while change_1d is True:  # loop as long as there is a significant on average
    for i in range(
        num_rep_1d
    ):  # repeat a certain number of times to make sure it ends at a local minimum
        if np.absolute(grad_diff / (i + 1)) >= 1e-1 and i == 0:
            grad = fn1d_prime(x_start_d1)
            ys.append(fn1d(x_start_d1))
            xs.append(x_start_d1)
            x_start_d1 = x_start_d1 - lr * grad  # optimize
        elif np.absolute(grad_diff / (i + 1)) >= 1e-1:
            grad = fn1d_prime(x_start_d1)
            grad_diff += grad if i != 0 else None
            ys.append(fn1d(x_start_d1))
            xs.append(x_start_d1)
            x_start_d1 = x_start_d1 - lr * grad  # optimize
        else:
            change_1d = False
    grad_diff = grad

### 1.1 Plot the Function

In [ ]:
# define range and values
xmin = -2
xmax = 2
x = np.arange(xmin, xmax, 0.01)
y = [fn1d(val) for val in x]

df = pd.DataFrame(dict(x=x, y=y))
fig = px.line(df, x="x", y="y", title="Line Plot")

fig.show()

### 1.2 Animate the process of Gradient descent 

In [ ]:
# define the range and the values
x = np.arange(xmin, xmax, 0.01)
y = [fn1d(val) for val in x]

# initialize the plot and the markers for gradient descent
fig = go.Figure(
    data=[
        go.Scatter(
            x=x,
            y=y,
            mode="lines",
            line=dict(color="green", width=1),
        ),
        go.Scatter(
            x=[xs[0]],
            y=[ys[0]],
            mode="markers",
            marker=dict(color="red", size=10),
        ),
    ]
)

# update layout parameters and add start button for animation
fig.update_layout(
    width=1200,
    height=800,
    xaxis=dict(range=(np.min(x) - 1, np.max(x) + 1), autorange=False, zeroline=False),
    yaxis=dict(range=(np.min(y) - 1, np.max(y) + 1), autorange=False, zeroline=False),
    title_text="2D Gradient Descent Animation",
    # start button config
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    args=[
                        None,
                        {
                            "frame": {"duration": 500, "redraw": False},
                            "fromcurrent": True,
                            "transition": {"duration": 1000, "easing": "cubic"},
                        },
                    ],
                    label="start",
                    method="animate",
                )
            ],
        )
    ],
)

# specify the animation frames
fig.update(
    frames=[
        go.Frame(data=[go.Scatter(x=[xs[k]], y=[ys[k]])], traces=[1])
        for k in range(len(ys))
    ]
)

# show result
fig.show()

## 2. Gradientenverfahren 2D

#### 2.0.1 Define function and derivative

In [ ]:
# Funktion definieren
def fn2d(x, y) -> float:
    return np.sqrt(np.pow(x, 2) + np.pow(y, 2))


# Ableitung
def fn2d_prime(x: float, y: float) -> np.ndarray:  # gibt 2D Array zurück
    partial_x = 2 * x  # leite f nach x ab, lass y konstant
    partial_y = 2 * y  # leite f nach y ab, lass x konstant
    return np.array([[partial_x], [partial_y]], ndmin=2)

#### 2.0.2 Gradient Descent block

In [ ]:
# Lokales Minimum finden mit Gradientenverfahren
# Parameters
x_start = 2.0  # Startpunkt
y_start = 2.0
lr = 1e-1  # Schrittgröße
num_rep_2d = 10
grad_xy_diff = fn2d_prime(x_start, y_start)
change_2d = True
xs_2d = [x_start]
ys_2d = [y_start]
zs_2d = [fn2d(x_start, y_start)]

# same algorithm as in 1 dimensional gradient
while change_2d == True:
    for i in range(num_rep_2d):
        if np.absolute(np.mean(grad_xy_diff)) / (i + 1) >= 1e-3 and i == 0:
            grad_xy = fn2d_prime(x_start, y_start)
            x_start = x_start - lr * grad_xy[0, 0]
            y_start = y_start - lr * grad_xy[1, 0]
            zs_2d.append(fn2d(x_start, y_start))
            xs_2d.append(x_start)
            ys_2d.append(y_start)
        elif np.absolute(np.mean(grad_xy_diff)) / (i + 1) >= 1e-3:
            grad_xy = fn2d_prime(x_start, y_start)
            grad_xy_diff[0, 0] += grad_xy[0, 0]
            grad_xy_diff[1, 0] += grad_xy[1, 0]
            x_start = x_start - lr * grad_xy[0, 0]
            y_start = y_start - lr * grad_xy[1, 0]
            zs_2d.append(fn2d(x_start, y_start))
            xs_2d.append(x_start)
            ys_2d.append(y_start)
        else:
            change_2d = False
            zs_2d.append(fn2d(x_start, y_start))
            xs_2d.append(x_start)
            ys_2d.append(y_start)
    grad_xy_diff[0, 0], grad_xy_diff[1, 0] = grad_xy[0, 0], grad_xy[1, 0]
    # print(x_start, grad_xy[0, 0], y_start, grad_xy[1, 0])

### 2.1 Plot the Surface

In [ ]:
# pass this argument to make sure the output is rendering in your notebook
pio.renderers.default = "notebook"

# define range and meshgrid to let numpy handle the iteration through the arrays
xy_min = -2
xy_max = 2
r = np.arange(xy_min, xy_max, 0.01)
x, y = np.meshgrid(r, r)

# calculate the corresponding values of the function
vals = fn2d(x, y)

# initialize the figure of the plot with the data
fig2 = go.Figure(data=[go.Surface(z=vals, x=x, y=y)])

# update layout
fig2.update_layout(title="Surface Plot", autosize=True)

# show result
fig2.show()

### 2.2 Plot the process of Gradient Descent

In [ ]:
# make sure the output renders
pio.renderers.default = "notebook"

# define range, meshgrid and values
r = np.arange(xy_min, xy_max, 0.01)
x, y = np.meshgrid(r, r)

vals = fn2d(x, y)

# initialize figure with Surface plot and markers
fig = go.Figure(
    data=[
        go.Surface(z=vals, x=x, y=y),
        go.Scatter3d(
            x=[xs_2d[0]],
            y=[ys_2d[0]],
            z=[zs_2d[0]],
            mode="markers",
            marker=dict(color="cyan", size=5),
        ),
    ]
)

# configure layout
fig.update_layout(
    # define range of the different axes
    scene=dict(
        xaxis=dict(
            range=(np.min(x) - 2, np.max(x) + 2), autorange=False, zeroline=False
        ),
        yaxis=dict(
            range=(np.min(y) - 2, np.max(y) + 2), autorange=False, zeroline=False
        ),
        zaxis=dict(
            range=(np.min(zs_2d), np.max(zs_2d) + 2), autorange=False, zeroline=False
        ),
    ),
    title_text="3D Gradient Descent Animation",
    # animation configuration -> redraw needs to be true
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    args=[
                        None,
                        {
                            "frame": {"duration": 500, "redraw": True},
                            "fromcurrent": True,
                            "transition": {"duration": 500},
                        },
                    ],
                    label="start",
                    method="animate",
                )
            ],
        )
    ],
)

# handle animation frames
fig.update(
    frames=[
        go.Frame(
            data=[go.Scatter3d(x=[xs_2d[k]], y=[ys_2d[k]], z=[zs_2d[k]])], traces=[1]
        )
        for k in range(len(zs_2d))
    ]
)

# output the figure
fig.show()